Hii team @natalierand_21292  @neilbhatt  i was working on a fine tunning project and my credits got exhausted and I cannot even run the pod on CPU to get the code and my files on GitHub or local system 
I don't have a credit card can you plz help me with this its urgent! 

In [ ]:
%pip install pytesseract Pillow

# CREATE THE JSON FILE USING OCR 

In [ ]:
import os
import json
from PIL import Image
import pytesseract
import random

# Directory containing the images
image_dir = "dataset/train/"  # Path to your images

# Initialize the dataset list
dataset = []

# User instruction for all entries
instruction = "Extract the text from the provided image."

# Process each image in the directory
for image_file in os.listdir(image_dir):
    if image_file.endswith(".jpg"):  # Ensure we're only processing .jpg files
        image_path = os.path.join(image_dir, image_file)
        try:
            # Open the image using PIL
            pil_image = Image.open(image_path).convert("RGB")
            
            # Extract text using Tesseract OCR with PSM 3 (fully automatic page segmentation)
            extracted_text = pytesseract.image_to_string(pil_image, config='--psm 3')
            
            # Create the conversation entry in the required format
            conversation = [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": instruction},
                        {"type": "image", "image": image_path}
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": extracted_text}
                    ]
                }
            ]
            
            # Add the conversation to the dataset
            dataset.append(conversation)
            print(f"Processed {image_path}: {extracted_text[:50]}...")
            
        except Exception as e:
            print(f"Error processing {image_path}: {e}")


# Save the full dataset to a JSON file
with open("processed_dataset_V6.json", "w") as f:
    json.dump(dataset, f, indent=4)

print(f"Dataset creation completed.")
print(f"Full dataset saved to 'processed_dataset_V6.json' with {len(dataset)} entries.")

# Print the first entry as an example
if dataset:
    print("\nFirst entry example:")
    print(json.dumps(dataset[0], indent=2))
else:
    print("No entries in the dataset.")

# PUSH THE DATASET TO HF

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="dataset/processed_dataset_V6.json")

dataset

In [ ]:
dataset["train"][0]

In [ ]:
from huggingface_hub import  notebook_login
notebook_login()

In [ ]:
dataset.push_to_hub("Aditya-Khedekar/SarvamAI-VLM-dataset")

# LOAD THE DATASET

In [10]:
from datasets import load_dataset

dataset = load_dataset("Aditya-Khedekar/SarvamAI-VLM-dataset")

dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'text'],
        num_rows: 107
    })
    test: Dataset({
        features: ['image', 'text'],
        num_rows: 29
    })
})

# MERGE AND PUSH THE MODEL TO HF 
- Unfortunately this model is not merged properly :( (https://huggingface.co/Aditya-Khedekar/SarvamAI-VLM)
- So you will have to push the (lora_model_V2) folder to drive and mount the drive to colab copy the path and then run the 

In [ ]:
%pip install -q -U peft

In [ ]:
from transformers import MllamaForConditionalGeneration, AutoTokenizer
from peft import PeftModel
from huggingface_hub import HfApi, login

# Step 1: Load the base model with the correct class
base_model_name = "unsloth/Llama-3.2-11B-Vision-Instruct"
base_model = MllamaForConditionalGeneration.from_pretrained(base_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Step 2: Load the adapters
adapter_path = "lora_model_V2"  # Path to your adapter directory
model = PeftModel.from_pretrained(base_model, adapter_path)

# Step 3: Merge the adapters with the base model
merged_model = model.merge_and_unload()

# Step 4: Save the merged model locally
merged_model_path = "merged_model_V2"
merged_model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)  # Save the tokenizer too

# Step 5: Push the merged model to Hugging Face
# disabled token
login(token="")
repo_id = "Aditya-Khedekar/SarvamAI-VLM"  # Your repository name
api = HfApi()
api.upload_folder(
    folder_path=merged_model_path,
    repo_id=repo_id,
    repo_type="model",
)
print(f"Merged model successfully uploaded to https://huggingface.co/{repo_id}")